In [1]:
import numpy as np

In [15]:
from IPython.core.debugger import set_trace
import numpy as np

# This program mimics the MatSPL representation in Spiral
# General Design:
#     Everything is a Mat
#     Mats are evaluated using getMat, which is done as late as possible to to maximize flexibility
#     There are 4 base Mats (NTT,TW,L,I) which correspond to the Spiral definitions
#     There are 2 binary Mat operations (MM (Matrix Multiply), Tensor) which are themselves Mats

class Mat:
    def __init__(self,n,p):
        self.n=n
        self.p=p
    #returns nxn matrix
    def getMat(self):
        pass
    def __str__(self):
        return "{}\n".format(str(self.getMat()))

class NTT(Mat):
    def __init__(self,pr,I=False,T=False):
        Mat.__init__(self,pr.n,pr.p)
        self.I=I
        self.T=T
        self.NW=pr.Pos
        self.pr=pr
        self.p=pr.p
        self.n=pr.n
        self.psi,self.w=pr.getRoots(I)
    def getMat(self):
        ret = self.term_psi() if self.NW else self.term()
        return ret.T if self.T else ret
    def term(self):
        return np.array([[pow(self.w,i*j,self.p) for i in range(self.n)] for j in range(self.n)])
    def term_psi(self):
        return np.array([[pow(self.w,i*j,self.p)*pow(self.psi,i,self.p)%self.p for i in range(self.n)] for j in range(self.n)])

    
class Tw(Mat):
    def __init__(self,pr,m,I=False):
        Mat.__init__(self,pr.n,pr.p)
        assert n%m==0, "n:%s m:%s"%(n,m)
        self.NW=pr.Pos
        self.m=m
        self.psi,self.w=pr.getRoots(I)
    def getMat(self):
        vec=[]
        for i in range(self.n//self.m):
            for j in range(self.m):
                if self.NW:
                    vec.append((pow(self.w,i*j,self.p)*pow(self.psi,((1-self.m)*i)%(2*n),self.p))%self.p)
                else:
                    vec.append((pow(self.w,i*j,self.p)))
        ret = np.array([[vec[i] if i==j else 0 for i in range(self.n)] for j in range(self.n)])
        return ret

class Tw2(Tw):
    def __init__(self,pr,m,I=False):
        Tw.__init__(self,pr,m,I)
    def getMat(self):
        vec=[]
        for i in range(self.n//self.m):
            for j in range(self.m):
                if self.NW:
                    vec.append(pow(self.w,i*j,self.p)%self.p)
                else:
                    vec.append((pow(self.w,i*j,self.p)))
        ret = np.array([[vec[i] if i==j else 0 for i in range(self.n)] for j in range(self.n)])
        return ret

class Tw(Mat):
    def __init__(self,pr,m,I=False):
        self.p=pr.p
        self.n=pr.n
        self.m=m
        self.psi,self.w=pr.getRoots(I)
    def func(self,i,j):
        pass
    def getMat(self):
        vec=[]
        for i in range(self.n//self.m):
            for j in range(self.m):
                vec.append(self.func(i,j))
        ret = np.array([[vec[i] if i==j else 0 for i in range(self.n)] for j in range(self.n)])
        return ret
    
class Tw_CT(Tw):
    def __init(self,pr,m,I=False):
        Tw.__init__(self,pr,m,I)
        self.NW=pr.Pos
    def func(self,i,j):
         if self.NW:
            return (pow(self.w,i*j,self.p)*pow(self.psi,((1-self.m)*i)%(2*n),self.p))%self.p
         else:
            return pow(self.w,i*j,self.p)
        
class Tw_GS(Tw):
    def __init__(self,pr,m,I=False):
        Tw.__init__(self,pr,m,I)
        self.NW=pr.Pos
    def func(self,i,j):
        if self.NW:
            return pow(self.psi,i+2*i*j,self.p)
        else:
            return pow(self.w,i*j,self.p)
        
class Tensor(Mat):
    def __init__(self,m1,m2):
        Mat.__init__(self,m1.n*m2.n,m1.p)
        assert m1.p==m2.p
        self.m1=m1
        self.m2=m2
    def getMat(self):
        m1m=self.m1.getMat()
        m2m=self.m2.getMat()
        ret = [[0 for i in range(self.n)] for j in range(self.n)]
        for i in range(self.n):
            for j in range(self.n):
                ret[i][j]=m1m[i//m2m.shape[0]][j//m2m.shape[0]]*m2m[i%m2m.shape[0]][j%m2m.shape[0]]%self.p
        return np.array(ret)

class MM(Mat):
    def __init__(self,m1,m2):
        Mat.__init__(self,m1.n,m1.p)
        assert m1.n==m2.n and m1.p==m2.p
        self.m1=m1
        self.m2=m2
    def getMat(self):
        ret = self.m1.getMat()@self.m2.getMat()
        return np.array([[ret[j][i]%self.p for i in range(self.n)] for j in range(self.n)])
    
class I(Mat):
    def __init__(self,n,p):
        Mat.__init__(self,n,p)
    def getMat(self):
        return np.array([[1 if i==j else 0 for i in range(self.n)] for j in range(self.n)])

class L(Mat):
    def __init__(self,n,p,str):
        Mat.__init__(self,n,p)
        self.str=str
        self.m=n//str
    def perm(self,i):
        return i//self.m+self.str*(i%self.m)
    def getMat(self):
        return np.array([[1 if self.perm(j)==i else 0 for i in range(self.n)] for j in range(self.n)])


class PolyRing:
    def __init__(self,n,p,Pos=False):
        self.n=n
        self.p=p
        self.Pos=Pos
        assert (p-1)%n==0
        k=(p-1)//n
        r=primitive_root(p)
        self.psi=pow(r,k//2,p)
        self.w=pow(r,k,p)
        self.psi_i = pow(self.psi,2*n-1,p)
        self.w_i = pow(self.w,n-1,p)
        assert self.psi*self.psi_i % p == 1
        assert self.w*self.w_i % p == 1
    def __str__(self):
        return f"PolyRing Z/{self.p}[X]/(X^{self.n} {'+' if self.Pos else '-'} 1)\n "
    def getRoots(self,I):
        if I:
            return self.psi_i,self.w_i
        else:
            return self.psi,self.w
    def reduce(self,m):
        assert self.n%m==0
        return PolyRing(self.n//m,self.p,self.Pos)
    def toNeg(self):
        return PolyRing(n,p,Pos=False)
    def inverse(self,a):
        return pow(a,p-2,p)
#some helper functions
def equal(m1,m2):
    if m1.n!=m2.n or m1.p!=m2.p:
        return False
    m1m=m1.getMat()
    m2m=m2.getMat()
    for i in range(m1.n):
        for j in range(m1.n):
            if m1m[i][j]!=m2m[i][j]:
                return False
    return True

def transform(m,v):
    mm=m.getMat()
    ret = mm*v
    return [i%m.p for i in ret]


def primitive_root(p):
    for i in range(1,p-1):
        eq1=False
        a=1
        for j in range(p-2):
            a=(a*i)%p
            if a==1:
                eq1=True
                break
        if not eq1:
            return i
    return -1

def is_prime(x):
    for i in range(2,int(x**1/2)+1):
        if x%i==0:
            return False
    return True

def getMod(n,NW=False):
    c=n
    if NW:
        c*=2
    while (not is_prime(c+1)):
        c+=2*n
    return c+1




In [3]:
pr = PolyRing(8,17,Pos=True)
print(pr)
ntt=NTT(pr)
print(ntt)
ntt_i=NTT(pr,I=True,T=True)
print(ntt_i)

PolyRing Z/17[X]/(X^8 + 1)
 
[[ 1  3  9 10 13  5 15 11]
 [ 1 10 15 14  4  6  9  5]
 [ 1  5  8  6 13 14  2 10]
 [ 1 11  2  5  4 10  8  3]
 [ 1 14  9  7 13 12 15  6]
 [ 1  7 15  3  4 11  9 12]
 [ 1 12  8 11 13  3  2  7]
 [ 1  6  2 12  4  7  8 14]]

[[ 1  1  1  1  1  1  1  1]
 [ 6 12  7 14 11  5 10  3]
 [ 2  8 15  9  2  8 15  9]
 [12 11  3  7  5  6 14 10]
 [ 4 13  4 13  4 13  4 13]
 [ 7  3 11 12 10 14  6  5]
 [ 8  2  9 15  8  2  9 15]
 [14  7 12  6  3 10  5 11]]



In [9]:
#functions to define and test transforms

class Reduction:
    def __init__(self,mats,par):
        self.mats=mats
        self.p=par.p
        self.par=par
    def multiply(self,x):
        for i in range(len(self.mats)-1,-1,-1):
            print(x)
            x=self.mats[i].getMat()@x
            for j in range(len(x)):
                x[j]%=self.p
        return x   
    def result(self):
        if len(self.mats)==0:
            return "no mats"
        ret=self.mats[0]
        for i in range(1,len(self.mats)):
            ret=MM(ret.copy(),self.mats[i])
        return ret
    def __str__(self):
        ret=""
        for m in self.mats:
            ret+=str(m)
        return ret
    def test_transform(self):
        t_res=self.mats[0].getMat()
        for i in range(1,len(self.mats)):
            t_res=t_res@self.mats[i].getMat()
        t_res%=self.p
        print("reduction:\n%s"%t_res)
        print("parent:\n%s"%self.par.getMat())
        return t_res==self.par.getMat()
    
class CT(Reduction):
    def __init__(self,ntt,radix):
        Reduction.__init__(self,[],ntt)
        m=ntt.n//radix
        m1=Tensor(NTT(ntt.pr.reduce(m),I=ntt.I),I(m,p))
        m2=Tw_CT(pr,m,I=ntt.I)
        m3=Tensor(I(radix,p),NTT(ntt.pr.reduce(radix),I=ntt.I))
        m4=L(n,p,radix)
        self.mats=[m1,m2,m3,m4]

class GS(Reduction):
    def __init__(self,ntt,radix):
        Reduction.__init__(self,[],ntt)
        m=ntt.n//radix
        m1=L(n,p,radix)
        m2=Tensor(I(m,p),NTT(ntt.pr.reduce(m),I=ntt.I,T=True))
        m3=Tw_GS(pr,radix,I=ntt.I)
        m4=Tensor(NTT(ntt.pr.neg().reduce(radix),I=ntt.I,T=True),I(radix,p))
        self.mats=[m1,m2,m3,m4]

def convolution(pr,x,y):
    ret=[]
    for i in range(pr.n):
        a=sum(x[j]*y[i-j] for j in range(i))
        b=sum(x[j]*y[pr.n+i-j] for j in range(i+1,pr.n))
        c = (a-b)%pr.p if pr.Pos else (a+b)%pr.p
        ret.append(c)
    return np.array(ret)

def Mv(pr,m,v):
    return np.mod(m.getMat()@v,pr.p)

In [14]:
#test of convolution
pr=PolyRing(2,5,Pos=False)
ntt=NTT(pr,I=False)
ntt_i=NTT(pr,I=True)
x=np.arange(pr.n)
print(x)
print(ntt.getMat())
X = Mv(pr,ntt,x)
print(X)
X2 = X*X
print(X2)
X2on = X2*pow(pr.n,pr.n-1,pr.p)

[0 1]
[[1 1]
 [1 4]]
[1 4]
[ 1 16]
[1 1]


In [ ]:
n=4
p=17
radix=2
pr=PolyRing(n,p,Pos=True)
ntt=NTT(pr,I=False,T=True)
gs=GS(ntt,radix)
print(gs)
x=np.arange(n)
print(gs.multiply(x))
print(gs.test_transform())

In [ ]:
n=6
radix=3
p=getMod(n,NW=True)

print(p)
ntt=NTT(n,p,NW=True,Tr=True)
print(ntt.psi)
ct=GS_NW(ntt,radix)
print(ct)
x=np.arange(n)
print(ct.multiply(x))
print(ct.test_transform())

In [ ]:
#radix-n/m FFT for NTT

#setup
n=8
m=4
p=getMod(n)
p=17
print(p)

#transform matrices
m1=Tensor(NTT(n//m,p),I(m,p))
m2=Tw(n,p,m)
m3=Tensor(I(n//m,p),NTT(m,p))
m4=L(n,p,n//m)
print(m1)
print(m2)
print(m3)
print(m4)

#multiply and check result
res=MM(MM(m1,m2),MM(m3,m4))
print(res)
print(NTT(n,p))
print("FFT result is equivalent:")
print(equal(res,NTT(n,p)))